In [108]:
''' IMPORT ALL THE THINGS '''
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19


In [109]:
''' DEFINE PATH TO THE DATASETS '''

CWD = os.getcwd().replace('\\','/') # get current workspace directory
IMAGE_DIR = f'{CWD}/images'
TRAIN_DATASET_DIR = f'{IMAGE_DIR}/train'
TEST_DATASET_DIR = f'{IMAGE_DIR}/test'
VAL_DATASET_DIR = f'{IMAGE_DIR}/validation'

CATEGORIES = ['clementine','grapefruit','orange']

# print('CWD :',CWD)
# print('IMAGE_DIR :',IMAGE_DIR)
# print('TRAIN_DATASET_DIR :',TRAIN_DATASET_DIR)
# print('TEST_DATASET_DIR :',TEST_DATASET_DIR)
# print('VAL_DATASET_DIR :',VAL_DATASET_DIR)
# print('CATEGORIES :', CATEGORIES)

In [110]:
''' LOAD DATA into Variables '''

IMAGE_RESIZE = (224, 224)

print('-----------Train-----------')
#Load train data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DATASET_DIR, # train_data path
    image_size=IMAGE_RESIZE, # each image resize
    batch_size=3,
)

print(train_dataset.class_names)  # class number check
print(train_dataset)


print("-----------Validation-----------")
#Load validation data
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DATASET_DIR,  # validation_data path
    image_size=IMAGE_RESIZE,  # each image resize
    batch_size=3,
)
print(valid_dataset.class_names)  # class check
print(valid_dataset)


-----------Train-----------
Found 3 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
-----------Validation-----------
Found 3 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>


In [111]:
''' Create Augmented Data '''

' Create Augmented Data '

In [112]:
''' Get pre train model '''
base_model = VGG16()

In [113]:
''' Create a model '''
# new last layer
prediction_layer = layers.Dense(3, activation='softmax', name="prediction")(base_model.get_layer(index=-1).output)

# create a model from pre train model with new last layer
model = Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary() # show all the model's layer

In [115]:
''' Compiling '''

opt = tf.keras.optimizers.Adam(learning_rate=0.001) ### can be changed
los = tf.keras.losses.BinaryCrossentropy()  ### can be changed
met = tf.keras.metrics.Accuracy()  ### can be changed

model.compile(optimizer=opt, loss=los, metrics=met)


In [ ]:
''' Training '''
model.fit(
    train_dataset, # both data and label cause we use ImageDataGenerator
    validation_data=valid_dataset,
    epochs=10, ### can be change
    shuffle=True 
)


In [ ]:
''' Evaluating '''

In [ ]:
''' Predicting '''